In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,Flatten,LSTM
from tensorflow.keras.utils import to_categorical as to_cat
import numpy as np ; import matplotlib.pyplot as plt ;  import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import warnings,re ,string
warnings.filterwarnings("ignore")

2025-06-21 08:40:34.490694: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750495234.682100      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750495234.731705      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data =  pd.read_csv("/kaggle/input/top-250-korean-dramas-kdrama-dataset/kdrama.csv")["Synopsis"][:50]
print("Shape of the data : ",data.shape)
print(data[:3])

Shape of the data :  (50,)
0    Geu Roo is a young autistic man. He works for ...
1    Although Baek Hee Sung is hiding a dark secret...
2    The stories of people going through their days...
Name: Synopsis, dtype: object


In [3]:
def remove_mentions(text):
    text = re.sub(r'@\S*', ' ', text)
    text = re.sub(r'#\S*', ' ', text)
    return text

def removing_special_chars(text):
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
    return re.sub(pat,' ',text)


def remove_punctuation(text):
    return ''.join([c for c in text if c not in string.punctuation])

def clean_text(text):
    text =  str(text).lower()
    text =  remove_mentions(text)
    text  =  removing_special_chars(text)
    text =  remove_punctuation(text)
    return text

In [4]:
for index,i in enumerate(data) :
    data[index] = clean_text(i)   # Clean text
     
token  = Tokenizer()
token.fit_on_texts(data.ravel())   # ravel for fitting it u can use flatten also 


### Look at down what is the stratergy we are applying  here is :

In [5]:
sample = []
    
tokenized =  token.texts_to_sequences([data[0]])[0]
for i in range(1,len(data[0])):
    sample.append(tokenized[:i+1])
sample[:5]

[[116, 169],
 [116, 169, 7],
 [116, 169, 7, 2],
 [116, 169, 7, 2, 21],
 [116, 169, 7, 2, 21, 471]]

### Lets Look at the logic what is happening 

=> suppose a sample sentence `This sent is good`  so we will make it a supervised learning problem by 


| Text    | Ouput |
| -------- | ------- |
| This  | sent    |
| This sent | is     |
| This sent is     | good    |

<u> **we will create a vector like this** </u> :
```
[116, 169],  =>  116 |  169
 [116, 169, 7], =>  116,169 | 7
 [116, 169, 7, 2], => 116 169 7| 2
 [116, 169, 7, 2, 21], => 116 169 7 2 | 21
 [116, 169, 7, 2, 21, 471]] = > 116 169 7 2 21| 471
```

In [6]:
input_seqs = []
for sent in data.ravel():
    
    tokenized =  token.texts_to_sequences([sent])[0]
    for i in range(1,len(sent)):
        input_seqs.append(tokenized[:i+1])
        
max_length =  max([len(x)  for x in input_seqs])
padded =  pad_sequences(input_seqs,maxlen=max_length)

**In the above cell started with range(1,..) not range(0,... )?** <br />
=>  cuz we are talking two words initally so later first word can become `text`  and second become `output` <br />
**Example**:  we wil start with `[116,169]` then it becomes `116 |119`


In [7]:
X =  padded[:,:-1]
Y =  padded[:,-1]
Y =  to_cat(Y,num_classes=len(token.word_index) + 1) # classes start with 0

In [8]:
print("Num words (vocab size ): ",len(token.word_index))
print("Max Length : ",max_length)

Num words (vocab size ):  1291
Max Length :  187


In [9]:
print("Shape x :",X.shape)
print("Shape Y ",Y.shape)

Shape x : (20114, 186)
Shape Y  (20114, 1292)


**`Embedding(num_words,output_shape,max_length_of_input)`** <br />
=> `num_words` = `num_classes` of y =  length_of_`word_index+1`


In [10]:
model =  Sequential([
    Embedding(len(token.word_index) + 1,100,input_length=max_length),
    LSTM(256),
    Dense(len(token.word_index) + 1,activation="softmax")
    
])
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
model.summary()

I0000 00:00:1750495249.440328      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
history =  model.fit(X,Y,epochs=20)

Epoch 1/20


I0000 00:00:1750495253.659989      59 cuda_dnn.cc:529] Loaded cuDNN version 90300


629/629 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - acc: 0.3613 - loss: 3.8320
Epoch 2/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.8307 - loss: 1.1153
Epoch 3/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.8330 - loss: 1.0233
Epoch 4/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - acc: 0.8411 - loss: 0.8843
Epoch 5/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.8465 - loss: 0.7656
Epoch 6/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.8615 - loss: 0.6274
Epoch 7/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.8914 - loss: 0.4829
Epoch 8/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9261 - loss: 0.3491
Epoch 9/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9538 - loss: 0.2421
Epoch 10/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9740 - loss: 0.1611
Epoch 11/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9865 - loss: 0.1041
Epoch 12/20
629/629 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - acc: 0.9946 - loss: 0.0608
Epoch 13/20
629/629 ━━━

In [12]:
text  = "seoul"

for J in range(20):
    text_Seq = pad_sequences(token.texts_to_sequences([text]),maxlen=max_length)
    verbs = np.argmax(model.predict(text_Seq))
    for key,value in token.word_index.items():
        if value == verbs :
           text =  text + " " +key
           print(text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
seoul the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
seoul the events
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
seoul the events in
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
seoul the events in season
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
seoul the events in season one
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
seoul the events in season one waves
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
seoul the events in season one waves of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
seoul the events in season one waves of the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
seoul the events in season one waves of the undead
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
seoul the events in season one waves of the undead threaten
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
seoul the events in season one waves of the undead threaten to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
seoul the events in season one waves of the undead threaten to reach
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
seoul the events in season one waves